In [1]:
# imports 

import cv2
import mediapipe as mp

In [2]:
# --- 1. Initialization ---

# Initialize MediaPipe Hands solution
# mp.solutions.hands contains the API for the hand tracking model
mp_hands = mp.solutions.hands 
# mp.solutions.drawing_utils contains the utility to draw lines and points
mp_drawing = mp.solutions.drawing_utils 

# Start video capture (0 is usually the default camera)
cap = cv2.VideoCapture(0)

In [ ]:


# Configure the Hand model
# 'static_image_mode=False': Optimize for video streaming (faster tracking)
# 'max_num_hands=1': Only track one hand for simplicity in this initial phase
# 'min_detection_confidence=0.5': Minimum confidence value ([0.0, 1.0]) for the detection to be considered successful
# 'min_tracking_confidence=0.5': Minimum confidence value for the tracking to be maintained
with mp_hands.Hands(
        static_image_mode=False,
        max_num_hands=1,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as hands:
    
    # --- 2. Main Processing Loop ---
    
    while cap.isOpened():
        # Read a frame from the camera
        success, image = cap.read()
        
        # If frame read failed, break the loop
        if not success:
            print("Ignoring empty camera frame.")
            continue
            
        # Optimization: Convert the image from BGR (OpenCV default) to RGB (MediaPipe requirement)
        # and set it to non-writable for performance.
        image.flags.writeable = False
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Process the image to find hands and landmarks
        results = hands.process(image)
        
        # Convert the image back to BGR for display purposes
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # --- 3. Landmark Drawing and Extraction ---

        # 'results.multi_hand_landmarks' is a list of detected hands (if any)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                
                # Draw the 21 hand landmarks and connections on the image
                mp_drawing.draw_landmarks(
                    image, 
                    hand_landmarks, 
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2),  # Landmark color (Red)
                    mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2)  # Connection color (Green)
                )
                
                # --- Important: Data Extraction Example ---
                # To use the coordinates in Phase 2, you need to access them like this:
                
                # The image dimensions are needed to convert normalized coordinates to pixel values
                h, w, c = image.shape  # height, width, channels
                
                # Get the coordinates for the Index Finger Tip (Landmark 8)
                # Landmark indices: 0=Wrist, 4=Thumb Tip, 8=Index Tip, 12=Middle Tip, 16=Ring Tip, 20=Pinky Tip
                
                # 'lm' is a normalized coordinate (0.0 to 1.0)
                index_tip_lm = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                
                # Convert to pixel coordinates (integer values)
                index_tip_x = int(index_tip_lm.x * w)
                index_tip_y = int(index_tip_lm.y * h)
                
                # Draw a circle on the Index Finger Tip as a simple cursor visualization
                cv2.circle(image, (index_tip_x, index_tip_y), 10, (255, 0, 0), -1) # Blue circle
                
                # The coordinates (index_tip_lm.x, index_tip_lm.y, index_tip_lm.z) 
                # are what you will use for gesture recognition in Phase 2!
        
        # --- 4. Display and Exit ---
        
        # Display the processed image
        cv2.imshow('Hand Tracking Input System - Press Q to Exit', image)
        
        # Break the loop if the 'q' key is pressed
        if cv2.waitKey(5) & 0xFF == ord('q'):
            break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

e:\projects\hand_track_num_detect\venv\lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


KeyboardInterrupt: 

: 